In [1]:
# Parameters
msgs = "Ran from Airflow at 2022-07-09T04:00:00+00:00!"


In [2]:
from selenium import webdriver
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
driver = webdriver.Chrome('/usr/lib/chromium-browser/chromedriver',chrome_options=chrome_options)

/data/venv/lib/python3.6/site-packages/ipykernel_launcher.py:6: DeprecationWarning: use options instead of chrome_options
  


In [3]:
import time
import os
import requests
import urllib.request
import urllib.parse
import json
import re
import collections
import pandas as pd
import bs4
from bs4 import BeautifulSoup
import dateparser
from datetime import datetime
import numpy as np
import logging

from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By

In [4]:
ID = '004'
SITE ="xalimasn.com"

In [5]:
categories = ["presidentielle-2012","chroniques","blog","celebrites","sports"]

In [6]:
def parseDate(date):
    try:
        date = str(date)
        if date.split(" ")[-1] == "ago":
            date = date.replace('ans', "years").replace("an","years").replace("heures", "hours").replace("heure", "hour").replace("semaines", "weeks").replace("semaine", "weeks").replace("mois", "months").replace("secondes","seconds").replace("seconde","seconds").replace("jours","days").replace("jour","days")
            return dateparser.parse(str(date))
        else:
            return dateparser.parse(date, languages=['fr'])
    except:
        return dateparser.parse(date)

In [7]:
def getWebPage(URL):
  page  = requests.get(URL)
  return BeautifulSoup(page.text, "html.parser")

In [8]:
def getArticleTitle(HTML):       
  h = HTML.find_all("h1", {"class": "jeg_post_title"})
  return str(h[0].text).replace("\n","").replace("\t", "").replace("\r", "")

In [9]:
def getArticleContent(HTML):
  h = HTML.find_all("div", {"class": "content-inner"})[0].find_all("p")
  return " ".join([i.text for i in h])

In [10]:
def getArticleAuthor(HTML):
    try:
        h = HTML.find_all("h3", {"class": "jeg_author_name"})
        return str(h[0].text).replace("\n","").replace("\t", "").replace("\r", "")
    except:
        return ""

In [11]:
def getArticleDate(HTML):
  h = HTML.find_all("div", {"class": "jeg_meta_date"})
  return str(h[0].text).replace("\n","").replace("\t", "").replace("\r", "")


In [12]:
def getArticleCommentList(HTML):
    try:
        return "|".join([i.text for i in HTML.find_all('ol',{'class', 'commentlist'})[0].find_all('p')])
    except:
        return ""

In [13]:
def getNewLinks(category, checkPoint):
    driver = webdriver.Chrome('/usr/lib/chromium-browser/chromedriver',chrome_options=chrome_options)
    driver.get(f"https://www.xalimasn.com/rubrique/politique/{category}/")
    SCROLL_PAUSE_TIME = 2.5
    i = 0
    last_height = driver.execute_script("return document.body.scrollHeight")
    while True:
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(SCROLL_PAUSE_TIME)
        new_height = driver.execute_script("return document.body.scrollHeight")
        page =  BeautifulSoup(driver.page_source, "html.parser")
        _links = [i.find('a', href=True)['href'] for i in page.find_all("div",{"class":"jeg_main_content jeg_column col-sm-6"})[0].find_all("article")]
        if checkPoint[0] in _links or checkPoint[1] in _links:
            try:
                lastIndex = _links.index(checkPoint[0])
            except: lastIndex = _links.index(checkPoint[0])
            return _links[:lastIndex]
        if new_height == last_height:
            if i>3 :return _links
            else: i+=1
        last_height = new_height
        

        
        
def getArticle(link,category):
        try:
            p = getWebPage(link)
            _title      = getArticleTitle(p)
            _article    = getArticleContent(p)
            _date       = getArticleDate(p)
            _author     = getArticleAuthor(p)
            _commments  = getArticleCommentList(p)
            return [1,{'DATE_SCRAPING':str(datetime.today()),
                  'DATE':parseDate(_date),
                  'DATE_ARTICLE':_date,
                  'CATEGORY':category,
                  'URL':link,
                  "AUTEUR":_author,
                  "TITRE":_title,
                  "SITE":SITE,
                  "CONTENU":_article,
                  "COMMENTAIRES":_commments,}]
        except:
            return [0,link]


def updateJson(path, key, value):
    if not os.path.exists(path):
        _data = {key:[]}
    else:
        try:
            jsonFile = open(path, "r",encoding='utf-8')
            _data = json.load(jsonFile)
            jsonFile.close()
        except:
            print("error UpdateJson")
    _data[key].append(value)
    jsonFile = open(path, "w+", encoding ='utf-8')
    jsonFile.write(json.dumps(_data, default=str))
    jsonFile.close()
        
def saveArticle(document):
    try:
        Y = document["DATE"].year
        M = document["DATE"].month
    except:print(document)
    if not os.path.exists(f"/data/notebooks/DB/DATA/{Y}"):
        os.mkdir(f"/data/notebooks/DB/DATA/{Y}") 
    if not os.path.exists(f"/data/notebooks/DB/DATA/{Y}/{M}"):
        os.mkdir(f"/data/notebooks/DB/DATA/{Y}/{M}")
    return updateJson(f"/data/notebooks/DB/DATA/{Y}/{M}/DATA{ID}.json", "DATA", document)


def saveErrors(errList):
    path =  f"/data/notebooks/DB/ERRORLOGS/ERRLOGS{ID}.json"
    Y = datetime.today().year
    M = datetime.today().month
    try:
        jsonFile = open(path, "r",encoding='utf-8')
        _data = json.load(jsonFile)
        jsonFile.close()
    except:
        print("error in saveErrors")
        
    if f"{Y}{M}" not in _data: 
        _data[f"{Y}{M}"] = [errList]
    else:
        _data[f"{Y}{M}"].append(errList)
    jsonFile = open(path, "w+", encoding ='utf-8')
    jsonFile.write(json.dumps(_data, default=str))
    jsonFile.close()


def checkPoint(category, link=None):
    path = f"/data/notebooks/DB/CHECKPOINT/CP{ID}.json"
    try:
        jsonFile = open(path, "r",encoding='utf-8')
        _data = json.load(jsonFile)
        jsonFile.close()
    except:
        print("ERR")
    if link == None: return [_data[category]["cp1"], _data[category]["cp2"]]
    _data[category]["cp2"] = _data[category]["cp1"]
    _data[category]["cp1"] = link
    _data[category]["time"] = str(datetime.today())
    jsonFile = open(path, "w+", encoding ='utf-8')
    jsonFile.write(json.dumps(_data))
    jsonFile.close()
    
    
def update():
    error=[]
    for category in categories:
        _checkPoint = checkPoint(category)
        print(category)
        links = getNewLinks(category, _checkPoint)
        i = 0
        for link in links[::-1]:
            i+=1
            status, document = getArticle(link,category)
            if status==0:error.append(document); continue
            saveArticle(document)
            checkPoint(category,link)
    saveErrors(error)


def initCP():
    path = f"/data/notebooks/DB/CHECKPOINT/CP{ID}.json"
    _data = {}
    for c in categories:
        _data[c] =""
    jsonFile = open(path, "w+", encoding ='utf-8')
    jsonFile.write(json.dumps(_data))
    jsonFile.close()

In [ ]:
update()